### Must run step by step as there are command lines to be executed in the middle of the flow

### Input files: Price-Today.csv
### Output files: daily-sales.csv, daily-sales-prices.csv

In [3]:
import numpy as np
import os
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data"
csv_path = os.path.join(os.path.expanduser("~"), "iCloudDrive")
one_path = os.path.join(os.path.expanduser("~"), "OneDrive","Documents","Data")
osd_path = os.path.join(os.path.expanduser("~"),"OneDrive","Documents","obsidian-git-sync","Data")
dts_path = os.path.join(os.path.expanduser("~"),"Downloads","Datasets")

today = date.today()
today

datetime.date(2025, 1, 20)

In [5]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2025-01-20
yesterday: 2025-01-17


In [7]:
cols = 'name fm_date to_date fm_price to_price qty max_price min_price percent status'.split()

format_dict = {
    'fm_price':'{:.2f}','to_price':'{:.2f}','diff':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    'pct':'{:,.2f}%','percent':'{:,.2f}%',   
    'fm_date':'{:%Y-%m-%d}','to_date':'{:%Y-%m-%d}',
    'created_at':'{:%Y-%m-%d}','updated_at':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
}

In [9]:
sql = """
SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '%s'
ORDER BY a.name
"""
sql = sql % yesterday
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp.shape


SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '2025-01-17'
ORDER BY a.name



(20, 1)

In [11]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % yesterday
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '2025-01-17' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



### This statement causes program to hang when there is no data

In [14]:
df = pd.read_sql(sql, conlite)
#df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('diff = to_price - fm_price',inplace=True)
df['percent'] = round(df['diff']/df['fm_price']*100,2)
#df.style.format(format_dict)
df

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
0,3BBIF,2025-01-14,2025-01-17,5.10,5.10,12524587,5.15,5.05,B,SET,0.00,0.00
1,AIMIRT,2025-01-13,2025-01-17,10.50,10.50,1434700,10.60,10.40,B,SET,0.00,0.00
2,CPNREIT,2025-01-17,2025-01-17,12.20,12.20,1077970,12.20,12.10,B,SET,0.00,0.00
3,DIF,2025-01-16,2025-01-17,8.25,8.30,12414982,8.35,8.20,B,SET,0.05,0.61
4,GVREIT,2025-01-10,2025-01-17,6.55,6.65,1763967,6.65,6.45,B,SET,0.10,1.53
5,JMART,2025-01-16,2025-01-17,11.20,10.90,8907993,11.80,10.90,B,SET50,-0.30,-2.68
6,KCE,2025-01-17,2025-01-17,21.70,21.70,6559285,22.50,21.60,B,SET100,0.00,0.00
7,MCS,2025-01-08,2025-01-17,6.45,6.35,1052343,6.60,6.30,B,SET,-0.10,-1.55
8,NER,2025-01-17,2025-01-17,4.84,4.84,8175729,4.94,4.82,B,SET,0.00,0.00
9,SENA,2025-01-09,2025-01-17,2.20,2.20,1563360,2.24,2.16,B,SET,0.00,0.00


In [16]:
df.shape

(20, 12)

### IF the above count not equal number of orders, there must be something incorrect

### Create daily-sales from sales

In [20]:
df[cols].style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status
0,3BBIF,2025-01-14,2025-01-17,5.10,5.10,"12,524,587",5.15,5.05,0.00%,B
1,AIMIRT,2025-01-13,2025-01-17,10.50,10.50,"1,434,700",10.60,10.40,0.00%,B
2,CPNREIT,2025-01-17,2025-01-17,12.20,12.20,"1,077,970",12.20,12.10,0.00%,B
3,DIF,2025-01-16,2025-01-17,8.25,8.30,"12,414,982",8.35,8.20,0.61%,B
4,GVREIT,2025-01-10,2025-01-17,6.55,6.65,"1,763,967",6.65,6.45,1.53%,B
5,JMART,2025-01-16,2025-01-17,11.20,10.90,"8,907,993",11.80,10.90,-2.68%,B
6,KCE,2025-01-17,2025-01-17,21.70,21.70,"6,559,285",22.50,21.60,0.00%,B
7,MCS,2025-01-08,2025-01-17,6.45,6.35,"1,052,343",6.60,6.30,-1.55%,B
8,NER,2025-01-17,2025-01-17,4.84,4.84,"8,175,729",4.94,4.82,0.00%,B
9,SENA,2025-01-09,2025-01-17,2.20,2.20,"1,563,360",2.24,2.16,0.00%,B


In [22]:
file_name = "daily-sales.csv"
data_file = data_path + '/' + file_name
output_file = csv_path + '/' + file_name
one_file = one_path + '/' + file_name
osd_file = osd_path + '/' + file_name
print(data_file)

../data/daily-sales.csv


In [24]:
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(one_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(osd_file, header=True, index=False)

In [26]:
sales = df[cols]
sales.shape

(20, 10)

In [28]:
file_name = "Price-Today.csv"
input_file = data_path + '/' + file_name
prices = pd.read_csv(input_file)
prices.shape

(181, 10)

In [30]:
df_merge = pd.merge(sales,prices,on='name', how='inner')
df_merge.shape

(20, 19)

In [32]:
colu = 'name fm_date to_date fm_price to_price qty max_price min_price percent status \
price change volume date'.split()
df_merge[colu]

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status,price,change,volume,date
0,3BBIF,2025-01-14,2025-01-17,5.10,5.10,12524587,5.15,5.05,0.00,B,5.15,0.05,9872669,2025-01-20
1,AIMIRT,2025-01-13,2025-01-17,10.50,10.50,1434700,10.60,10.40,0.00,B,10.50,0.00,215148,2025-01-20
2,CPNREIT,2025-01-17,2025-01-17,12.20,12.20,1077970,12.20,12.10,0.00,B,11.90,-0.30,1842620,2025-01-20
3,DIF,2025-01-16,2025-01-17,8.25,8.30,12414982,8.35,8.20,0.61,B,8.35,0.05,3833113,2025-01-20
4,GVREIT,2025-01-10,2025-01-17,6.55,6.65,1763967,6.65,6.45,1.53,B,6.55,-0.10,364106,2025-01-20
5,JMART,2025-01-16,2025-01-17,11.20,10.90,8907993,11.80,10.90,-2.68,B,10.70,-0.20,3452811,2025-01-20
6,KCE,2025-01-17,2025-01-17,21.70,21.70,6559285,22.50,21.60,0.00,B,22.00,0.30,8274484,2025-01-20
7,MCS,2025-01-08,2025-01-17,6.45,6.35,1052343,6.60,6.30,-1.55,B,6.30,-0.05,232376,2025-01-20
8,NER,2025-01-17,2025-01-17,4.84,4.84,8175729,4.94,4.82,0.00,B,4.94,0.10,10645401,2025-01-20
9,SENA,2025-01-09,2025-01-17,2.20,2.20,1563360,2.24,2.16,0.00,B,2.18,-0.02,184331,2025-01-20


In [34]:
file_name = "daily-sales-prices.csv"
data_file = data_path + '/' + file_name
output_file = csv_path + '/' + file_name
one_file = one_path + '/' + file_name
osd_file = osd_path + '/' + file_name
output_file

'C:\\Users\\PC1\\iCloudDrive/daily-sales-prices.csv'

In [36]:
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(output_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(data_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(one_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(osd_file, header=True, index=False)

### Add or update sells record depends on trend

In [38]:
# Get the current working directory
current_dir = os.getcwd()
print(f"Current Directory: {current_dir}")

Current Directory: C:\Users\PC1\OneDrive\A5\Daily


In [40]:
# Derive the base directory (to_dir) by removing the last folder ('Daily')
to_dir = os.path.dirname(current_dir)
print(f"Base Directory (to_dir): {to_dir}")

Base Directory (to_dir): C:\Users\PC1\OneDrive\A5


In [42]:
os.chdir(to_dir)
%pwd

'C:\\Users\\PC1\\OneDrive\\A5'

In [44]:
!ruby ruby\\daily-out-new.rb

Name      From Date    To Date   From     To     Pct      Shares    Max    Min S Action
---------------------------------------------------------------------------------------
3BBIF    2025-01-14 2025-01-20   5.10   5.15   0.98%  12,524,587   5.15   5.05 B Update
AIMIRT   2025-01-13 2025-01-20  10.50  10.50    0.0%   1,434,700  10.60  10.40 B Update
AWC      2025-01-17 2025-01-20   3.14   3.04  -3.18%  55,878,974   3.18   3.04 S Update
BCH      2025-01-16 2025-01-20  14.60  14.20  -2.74%   5,779,848  14.90  14.20 S Update
CPNREIT  2025-01-17 2025-01-20  12.20  11.90  -2.46%   1,077,970  12.20  11.90 B Update
DIF      2025-01-16 2025-01-20   8.25   8.35   1.21%  12,414,982   8.35   8.20 B Update
GVREIT   2025-01-20 2025-01-20   6.65   6.55   -1.5%   1,763,967   6.65   6.45 B Insert
JMART    2025-01-16 2025-01-20  11.20  10.70  -4.46%   8,907,993  11.80  10.70 B Update
JMT      2025-01-16 2025-01-20  14.90  14.30  -4.03%  36,308,869  15.70  14.30 S Update
KCE      2025-01-17 2025-01-20  

In [45]:
os.chdir(current_dir)
%pwd

'C:\\Users\\PC1\\OneDrive\\A5\\Daily'



### cd\ruby\port_lite


### rails runner db/ins_sales.rb



In [48]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % today
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '2025-01-20' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



In [50]:
df = pd.read_sql(sql, conlite)
df.shape

(20, 10)

In [52]:
#df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('diff = to_price - fm_price',inplace=True)
df['percent'] = round(df['diff']/df['fm_price']*100,2)
df.style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
0,3BBIF,2025-01-14,2025-01-20,5.10,5.15,"25,049,174",5.15,5.05,B,SET,0.05,0.98%
1,AIMIRT,2025-01-13,2025-01-20,10.50,10.50,"2,869,400",10.60,10.40,B,SET,0.00,0.00%
2,CPNREIT,2025-01-17,2025-01-20,12.20,11.90,"2,155,940",12.20,11.90,B,SET,-0.30,-2.46%
3,DIF,2025-01-16,2025-01-20,8.25,8.35,"24,829,964",8.35,8.20,B,SET,0.10,1.21%
4,GVREIT,2025-01-20,2025-01-20,6.65,6.55,"1,763,967",6.65,6.45,B,SET,-0.10,-1.50%
5,JMART,2025-01-16,2025-01-20,11.20,10.70,"17,815,986",11.80,10.70,B,SET50,-0.50,-4.46%
6,KCE,2025-01-17,2025-01-20,21.70,22.00,"13,118,570",22.50,21.60,B,SET100,0.30,1.38%
7,MCS,2025-01-08,2025-01-20,6.45,6.30,"2,104,686",6.60,6.30,B,SET,-0.15,-2.33%
8,NER,2025-01-17,2025-01-20,4.84,4.94,"16,351,458",4.94,4.82,B,SET,0.10,2.07%
9,SENA,2025-01-09,2025-01-20,2.20,2.18,"3,126,720",2.24,2.16,B,SET,-0.02,-0.91%


In [54]:
df_sort = df.sort_values(by=['percent'],ascending=[True]).copy()

In [56]:
df_sort.query('percent <= -5.00').style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
18,TVO,2025-01-02,2025-01-20,22.90,21.10,"7,804,366",23.20,21.10,S,SET,-1.80,-7.86%


In [58]:
df_sort.query('percent >= 5.00').style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent


In [60]:
conlite.close()